In [1]:
import torch
from sentence_transformers import SentenceTransformer
from converter.converter import sentence_transformers_onnx

/home/estudiante/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# load the model
model = SentenceTransformer.load('./results/domain_adaptation_model/')

In [3]:
onnx_model = sentence_transformers_onnx(
    model,
    path = "triton/model_repository/domain_adaptation_model/1/model"
)
onnx_model

/home/estudiante/venv/lib/python3.10/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


In [4]:
onnx_model.encode("h")

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat1 in method wrapper_CUDA_addmm)

In [19]:
import numpy as np
from torchvision import transforms
from PIL import Image
import tritonclient.http as httpclient
from tritonclient.utils import triton_to_np_dtype

# Setting up client
client = httpclient.InferenceServerClient(url="localhost:8000")

inputs = httpclient.InferInput("input__0", (3,), datatype="FP32")
inputs.set_data_from_numpy(np.array([1,2,3], dtype='float32'), binary_data=True)

outputs = httpclient.InferRequestedOutput("output__0", binary_data=True, class_count=1000)

# Querying the server
results = client.infer(model_name="./results/domain_adaptation_model/", inputs=[inputs], outputs=[outputs])
inference_output = results.as_numpy('start')
print(inference_output[:5])

InferenceServerException: [404] Not Found